In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from pyod.models.knn import KNN 
import pyod
from pyod.utils.data import evaluate_print
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.font_manager
import numpy as np
from pyod.models.abod import ABOD
from pyod.models.knn import KNN
from pyod.utils.data import generate_data, get_outliers_inliers
import seaborn as sns 
from sklearn.neighbors import NearestNeighbors

In [3]:
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv('bsNET140513_032310.csv')

In [5]:
df.head()

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0


In [6]:
df['fraud'].value_counts()

0    587443
1      7200
Name: fraud, dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [8]:
df2=df.copy()

In [9]:
df2.loc[:,['typeTrans']]=le.fit_transform(df2.loc[:,['typeTrans']])
df2.loc[:,['Target']]=le.fit_transform(df2.loc[:,['Target']])
df2.loc[:,['Source']]=le.fit_transform(df2.loc[:,['Source']])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [9]:
data=df2[df2.columns.drop(['fraud'])].values

In [10]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.05)
yhat = iso.fit_predict(data)

In [11]:

data2=df2.values

In [12]:
mask = yhat == -1
outlier_values_if=pd.DataFrame(data2[mask, :])

In [13]:
outlier_values_if

,0,1,2,3,4
0,1372.0,0.0,105.59,4.0,0.0
1,408.0,0.0,159.92,4.0,0.0
2,723.0,34.0,324.50,4.0,1.0
3,2585.0,39.0,171.07,4.0,0.0
4,3999.0,9.0,36.41,0.0,0.0
...,...,...,...,...,...
29728,2646.0,0.0,258.61,4.0,0.0
29729,3864.0,31.0,71.83,2.0,0.0
29730,3864.0,11.0,26.48,2.0,0.0
29731,86.0,5.0,224.81,6.0,0.0


In [14]:
outlier_values_if[4].value_counts()

0.0    23735
1.0     5998
Name: 4, dtype: int64

In [10]:
from imblearn.under_sampling import RandomUnderSampler

In [11]:
from imblearn.over_sampling import RandomOverSampler


In [19]:
x=df2.drop('fraud',axis=1)
y=df2.fraud


In [21]:
y.value_counts()

0    587443
1      7200
Name: fraud, dtype: int64

In [27]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(x, y)

In [28]:
X_ros.shape

(1174886, 4)

In [29]:
y_ros.shape

(1174886,)

In [30]:
y_ros.value_counts()

1    587443
0    587443
Name: fraud, dtype: int64

In [33]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(x, y)

In [34]:
y_sm.value_counts()

1    587443
0    587443
Name: fraud, dtype: int64

In [31]:
from sklearn.svm import SVC,LinearSVC

In [45]:
train,test=train_test_split(df2,random_state=42)

In [46]:
train['fraud'].value_counts()

0    440532
1      5450
Name: fraud, dtype: int64

In [47]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(train.drop('fraud',axis=1),train.fraud)

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1))

In [26]:
x_train=scaling.fit_transform(X_sm)

In [48]:
xtest=test.drop('fraud',axis=1)
ytest=test.fraud

In [28]:
xtest=scaling.fit_transform(xtest)

In [14]:
# rus = RandomUnderSampler(random_state=42)
# X_rus, y_rus = rus.fit_resample(train.drop('fraud',axis=1),train.fraud)

In [30]:
y_sm.value_counts()

1    440532
0    440532
Name: fraud, dtype: int64

In [60]:
model = LinearSVC(class_weight='balanced')
model.fit(train.drop('fraud',axis=1),train.fraud)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(class_weight='balanced')

In [61]:
yhat = model.predict(xtest)

In [62]:
from sklearn.metrics import accuracy_score

In [63]:
accuracy_score(ytest,yhat)

0.9932665594876935

In [64]:
compare=pd.DataFrame()

In [65]:
compare['true']=np.array(ytest)

In [66]:
compare['preicted']=np.array(yhat)

In [67]:
compare['true'].value_counts()

0    146911
1      1750
Name: true, dtype: int64

In [68]:
len(compare[(compare.true==1) & (compare.preicted==1)])

792

In [42]:
from sklearn.metrics import confusion_matrix

In [43]:
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[1][1])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[0][0])

In [58]:
print_confusion_matrix(compare['true'],compare['preicted'])

True positive =  1481
False positive =  5665
False negative =  269
True negative =  141246


In [55]:
print_confusion_matrix(compare['true'],compare['preicted'])

True positive =  1448
False positive =  2789
False negative =  302
True negative =  144122


In [59]:
ytest.value_counts()

0    146911
1      1750
Name: fraud, dtype: int64

In [70]:
tn, fp, fn, tp = confusion_matrix(compare['true'],compare['preicted']).ravel()
print("tn, fp, fn, tp ---> ", tn, fp, fn, tp)

tn, fp, fn, tp --->  146868 43 958 792
